In [2]:
import pandas as pd 
import torch 

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset
import torch.optim as optim
from torch.optim.swa_utils import AveragedModel

import dask.dataframe as dd 
import numpy as np
import torch.nn as nn
import importlib
import pickle
import utils
import models
importlib.reload(utils)
from utils import *
importlib.reload(models)
from models import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
grouped_df = pd.read_csv("haha-longer-mid.csv")
print(grouped_df)

         game_id                                              moves  \
0       00VRM44j  e2e4 g8f6 e4e5 f6g8 f1b5 f7f6 e5e6 c7c6 e6d7 c...   
1       0MG5pz3L  c2c4 g7g6 e2e4 f8g7 d2d3 d7d6 f1e2 g8f6 g1f3 e...   
2       0MV7dmua  e2e4 e7e5 f2f4 e5f4 g1f3 d7d5 e4d5 d8d5 b1c3 d...   
3       0UNsyqEb  d2d4 d7d5 c2c4 g8f6 b1c3 c8f5 c1g5 e7e6 e2e3 h...   
4       10mypbCS  d2d4 g7g6 c1h6 f8h6 g1f3 g8f6 e2e3 b8c6 f1b5 d...   
...          ...                                                ...   
466483  yT5CKw8c  g1f3 e7e6 b1c3 d7d5 d2d4 g8f6 c1g5 c7c5 e2e3 h...   
466484  yxAYsxBu  e2e4 e7e5 d2d4 e5d4 c2c3 d4c3 f1c4 c3b2 c1b2 f...   
466485  zLOF4boA  d2d4 c7c5 d4c5 b8c6 a2a3 g7g6 g1f3 f8g7 c2c3 e...   
466486  zimmRKoi  e2e4 e7e5 g1f3 d7d6 f1c4 c7c6 h2h3 d8e7 e1g1 c...   
466487  zzidHYZH  e2e4 e7e6 d2d4 d7d5 e4e5 c7c5 c2c3 d8b6 g1f3 b...   

        white_elo  black_elo  white_active  \
0            1523       1500          True   
1            1538       1580          True   
2        

In [4]:
trainX_sequences, trainX, trainY, vocab = df_to_data(grouped_df, fixed_window=True, sampling_rate=0.25)
trainX_sequences, trainX_seqlengths  = pad_sequences(trainX_sequences)

In [5]:
trainX, trainX_sequences, trainX_seqlengths, trainY = np.array(trainX), np.array(trainX_sequences), np.array(trainX_seqlengths), np.array(trainY)

This is the last step of saving our data onto our disk so that it's ready to load as we train

In [8]:
def save_as_memmap(array, filename):
    # Determine the dtype and shape of the array to create a compatible memmap
    dtype = array.dtype
    shape = array.shape
    
    # Create a memmap file with write mode, which will also allocate the disk space
    memmap_array = np.memmap(filename, dtype=dtype, mode='w+', shape=shape)
    
    # Copy the data into the memmap array
    memmap_array[:] = array[:]
    
    # Flush memory changes to disk to ensure all data is written
    memmap_array.flush()

    # Return the path for confirmation
    return filename

# Save each array as a memmap file
filenames = [
    save_as_memmap(trainX_sequences, './../data/jan-march/trainX_sequences.memmap'),
    save_as_memmap(trainX, './../data/jan-march/trainX.memmap'),
    save_as_memmap(trainY, './../data/jan-march/trainY.memmap'),
    save_as_memmap(trainX_seqlengths, './../data/jan-march/trainX_seqlengths.memmap')
]


In [19]:
with open('./../data/jan-march/vocab.pkl', 'wb') as outp:
    pickle.dump(vocab, outp, pickle.HIGHEST_PROTOCOL)

Now that we've finished processing the data, let's now load in the data (assuming we're starting from fresh)

In [3]:
# Function to load a memmap file
def load_memmap(filename, dtype, shape):
    # Load the memmap file with read-only mode
    return np.memmap(filename, dtype=dtype, mode='r', shape=shape)

# Assuming you know the dtype and shape, for example:
# For trainX_sequences
dtype_trainX_sequences = np.int64  # or the correct dtype for your data
shape_trainX_sequences = (3038976, 16)  # replace with the correct shape
trainX_sequences = load_memmap('./../data/jan-march/trainX_sequences.memmap', dtype_trainX_sequences, shape_trainX_sequences)

# For trainX
dtype_trainX = np.int64  # or the correct dtype for your data
shape_trainX = (3038976, 12, 8, 8)  # replace with the correct shape
trainX = load_memmap('./../data/jan-march/trainX.memmap', dtype_trainX, shape_trainX)

# For trainY
dtype_trainY = np.int64  # or the correct dtype for your data
shape_trainY = (3038976,)  # replace with the correct shape
trainY = load_memmap('./../data/jan-march/trainY.memmap', dtype_trainY, shape_trainY)

# For trainX_seqlengths
dtype_trainX_seqlengths = np.int64  # or the correct dtype for your data
shape_trainX_seqlengths = (3038976,)  # replace with the correct shape
trainX_seqlengths = load_memmap('./../data/jan-march/trainX_seqlengths.memmap', dtype_trainX_seqlengths, shape_trainX_seqlengths)

In [4]:
with open('./../data/jan-march/vocab.pkl', 'rb') as inp:
    vocab = pickle.load(inp)

In [5]:
class MultimodalDataset(Dataset):
    def __init__(self, sequences, boards, lengths, labels):
        self.sequences = sequences
        self.boards = boards
        self.lengths = lengths
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.boards[idx], dtype=torch.float32),
            torch.from_numpy(self.sequences[idx], dtype=torch.long),
            torch.from_numpy(self.lengths[idx], dtype=torch.long),
            torch.from_numpy(self.labels[idx], dtype=torch.long),
        )
print(len(vocab.id_to_move.keys()))
print(torch.tensor(trainX[0]).shape)
print(trainX[0])

8121
torch.Size([12, 8, 8])
[[[0 0 0 0 0 0 0 0]
  [1 1 0 0 1 0 1 1]
  [0 0 1 0 0 1 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 1 0 0 0 0 1 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 1 0 0]
  [0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]



In [7]:
dataset = ChessDataset(trainX, trainY)
# Calculate split sizes
total_size = len(dataset)
print(total_size)
train_size = int(0.95 * total_size)
val_size = int(total_size * 0.04)

# Create subsets for training and validation
train_dataset = Subset(dataset, range(0, train_size))
val_dataset = Subset(dataset, range(train_size, train_size + val_size))
print(train_size)

3038976
